## 1. Get seizure templates based on other subjects to assess if seizures and baselines are alike

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from src import distance_utils as du
from src import level2_features as lf

patient_test = '413'

patient_train = ['400', '358', '352', '326', '365', '400', '413', '386']

list_features = ['meanrr', 'rmssd', 'nn50', 'pnn50', 'sdnn', 'hti', 'tinn', 'vlf_pwr', 'vlf_peak', 'vlf_rpwr', 'lf_pwr',
                 'lf_peak', 'lf_rpwr', 'lfnupwr', 'hf_pwr', 'hf_peak', 'hf_rpwr', 'hfnupwr', 'vhf_pwr', 'vhf_peak',
                 'vhf_rpwr', 'lf_hf', 's', 'sd1', 'sd2', 'sd12', 'sd21', 'hrmin', 'hrmax', 'hrminmax', 'hravg']

distances=['pearsonr', 'euclidean', 'cosine', 'chebyshev', 'braycurtis',
                                              'canberra', 'correlation', 'cityblock']

In [ ]:
pat1_level2features = pd.DataFrame()
i=0
for pat1 in patient_train:

    pat1_seizures = pd.read_parquet(f'..\data\clean_features_p{pat1}_seizure.parquet')
    pat1_seizures['range'] = pat1_seizures['time2sz']//39.

    i+=1
    for seizure in sorted(set(pat1_seizures['label'])):
        seiz_table = pat1_seizures.loc[pat1_seizures['label']==seizure]
        for feat in list_features:
            statistics = lf.statistics_(seiz_table[feat].values)
            statistics['label'] = seizure
            statistics['len'] = len(seiz_table)
            statistics['feature'] = feat
            pat1_level2features = pd.concat((pat1_level2features, pd.DataFrame(statistics, index=[0])), ignore_index=True)
    pat1_level2features.to_parquet(f'..\data\level2_features_p{pat1}_seizure.parquet')


In [ ]:
pat1 = '413'
pat1_seizures = pd.read_parquet(f'..\data\level2_features_p{pat1}_seizure.parquet')

In [ ]:
pat1_seizures.loc[pat1_seizures['feature'] == 'meanrr']

In [ ]:
import seaborn as sns
for feat in list_features:
    for pat in patient_train:
        pat_seizures = pd.read_parquet(f'..\data\clean_features_p{pat}_seizure.parquet')
        plt.figure()
        sns.lineplot(data = pat_seizures, x = 'time2sz', y = feat, label=feat)
        plt.legend()
        plt.show()